In [2]:
from google.colab import files
uploaded = files.upload()

Saving emails (1).zip to emails (1).zip


In [3]:
import zipfile
import os

with zipfile.ZipFile("emails (1).zip", 'r') as zip_ref:
    zip_ref.extractall("emails")  # Cria a pasta "emails" e extrai tudo dentro dela



# Caminhos corrigidos (agora com a pasta extra)
spam_path = "emails/emails/spam"
ham_path  = "emails/emails/ham"


In [7]:
# Importa os módulos necessários
import os          # Para manipular caminhos e diretórios
import io          # Para leitura de arquivos com codificações específicas
import numpy
import pandas as pd
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer  # Para converter texto em vetores numéricos
from sklearn.naive_bayes import MultinomialNB                # Algoritmo de classificação Naive Bayes


# Função para ler os arquivos de texto dos e-mails

def readFiles(path):
    # Percorre todos os arquivos dentro do diretório fornecido
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            filepath = os.path.join(root, filename)  # Monta o caminho completo do arquivo

            # Lê o arquivo linha por linha, procurando o corpo do e-mail (após a primeira linha em branco)
            inBody = False
            lines = []
            f = io.open(filepath, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)  # Lê apenas o corpo do e-mail
                elif line == '\n':
                    inBody = True  # O corpo começa após a primeira linha em branco
            f.close()

            # Junta as linhas do corpo em um único texto
            message = '\n'.join(lines)

            # Retorna o caminho do arquivo e a mensagem
            yield filepath, message


# Função para criar um DataFrame com os e-mails e sua classificação

def dataFrameFromDirectory(path, classification):
    rows = []   # Armazena os dados (mensagem + classe)
    index = []  # Armazena os nomes dos arquivos (como índice)

    # Lê todos os arquivos do diretório e armazena suas informações
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    # Retorna um DataFrame com os dados lidos
    return DataFrame(rows, index=index)


# Define os caminhos para os diretórios com os e-mails extraídos

# OBS: Esses caminhos foram ajustados após verificar a estrutura real dentro do .zip extraído
spam_path = "emails/emails/spam"  # Caminho para e-mails classificados como spam
ham_path  = "emails/emails/ham"   # Caminho para e-mails legítimos (ham)

# Cria os DataFrames separadamente
dataSpam = dataFrameFromDirectory(spam_path, "spam")
dataHam  = dataFrameFromDirectory(ham_path, "ham")

# Junta os dois em um único DataFrame se quiser trabalhar com tudo junto
data = pd.concat([dataSpam, dataHam])

#Aplicando treino/teste
#80% para treio e 20% para teste

dataSpamTrain = dataSpam.sample(frac=0.8, random_state=1) # random_state=1 para reprodutibilidade frac=0.8 significa 80% do dataset
dataSpamTest = dataSpam.drop(dataSpamTrain.index) # drop() remove os dados que estão no train set do test set

dataHamTrain = dataHam.sample(frac=0.8, random_state=1) # random_state=1 para reprodutibilidade frac=0.8 significa 80% do dataset
dataHamTest = dataHam.drop(dataHamTrain.index) # drop() remove os dados que estão no train set do test set

dataTrain = pd.concat([dataSpamTrain, dataHamTrain]) # concat() junta os dois dataframes, o de spam e o de ham, em um só dataframe de treino
dataTest = pd.concat([dataSpamTest, dataHamTest]) # concat() junta os dois dataframes, o de spam e o de ham, em um só dataframe de teste

In [8]:
data.head(25)

,message,class
emails/emails/spam/00369.845eeb9573484bd88a6a6224c7068d81,"<HR>\n\n<html>\n\n<div bgcolor=3D""#FFFFCC"">\n\...",spam
emails/emails/spam/00191.9ff80a41f015b7a6c409732e41c0df07,--374035298.1030784379296.JavaMail.SYSTEM.emas...,spam
emails/emails/spam/00120.58579af867ff9a702cff23e7b8818a59,This is a multi-part message in MIME format.\n...,spam
emails/emails/spam/00238.e3e16467d10137fa9a99b1701d76ae94,This is a multi-part message in MIME format.\n...,spam
emails/emails/spam/00121.bf18a63d6e7d40409f8b722036eadd82,"<html>\n\n\n\n<head>\n\n<meta http-equiv=3D""Co...",spam
emails/emails/spam/00337.813498483bc80a24c002e6e7e8e0f2cb,My name is Jeremy Lessaris. I'm looking for in...,spam
emails/emails/spam/00241.c28ade5771085a8fddd054a219566b7c,------=_NextPart_000_00B6_07E34C7A.C3030C43\n\...,spam
emails/emails/spam/00206.0c8362d7e86ddcaf39829800ac40e2ca,------=_NextPart_000_00E3_44B55B3D.C1001A02\n\...,spam
emails/emails/spam/00192.e5a6bb15ae1e965f3b823c75e435651a,This is a multi-part message in MIME format.\n...,spam
emails/emails/spam/00295.b028688ce4ea3693f4ed591b8ca3f72e,"<HTML>\n\n<BODY BGCOLOR=3D""#ffffff"">\n\n<P>\n\...",spam


In [9]:
dataTrain.head(25) #Exibe DataFrame de treino

,message,class
emails/emails/spam/00492.73db79fb9ad03aff1e08deb73b83203c,"<html><body><center>\n\n\n\n<table bgcolor=3D""...",spam
emails/emails/spam/00040.949a3d300eadb91d8745f1c1dab51133,Dear Sir or Madam:\n\n\n\nPlease reply to \n\n...,spam
emails/emails/spam/00220.cf7d03e161582887dc589229e2896e26,ATTN:\n\n\n\nI am Bulawa Mulete JR. the son of...,spam
emails/emails/spam/00207.0b71ac81a360455c1514f5872564b1e1,------=_NextPart_000_0044_5A8512D6.AD3AE071\n\...,spam
emails/emails/spam/00333.4bb36a535cb3d738f30f985f1e10a786,------000000000000000000000\n\nContent-Type: t...,spam
emails/emails/spam/00488.29e96da757cc5566c848833e26abdd65,"<HTML>\n\n<BODY>\n\n <tr valign=3D""top""> \n\n...",spam
emails/emails/spam/00002.d94f1b97e48ed3b553b3508d116e6a09,1) Fight The Risk of Cancer!\n\nhttp://www.adc...,spam
emails/emails/spam/00486.c0a2036a3da75d6d5ac6d19ab4b3d6ec,This is a multi-part message in MIME format.\n...,spam
emails/emails/spam/00090.52630c4c07cd069c7bc7658c1a7a7253,This is a multi-part message in MIME format.\n...,spam
emails/emails/spam/00314.8f7993db02bde4d724e1eff9d2d35db1,<html><head><title>::FREE MORTGAGE QUOTE::</ti...,spam


In [10]:
dataTest.head(25) #Exibe DataFrame de teste

,message,class
emails/emails/spam/00191.9ff80a41f015b7a6c409732e41c0df07,--374035298.1030784379296.JavaMail.SYSTEM.emas...,spam
emails/emails/spam/00238.e3e16467d10137fa9a99b1701d76ae94,This is a multi-part message in MIME format.\n...,spam
emails/emails/spam/00206.0c8362d7e86ddcaf39829800ac40e2ca,------=_NextPart_000_00E3_44B55B3D.C1001A02\n\...,spam
emails/emails/spam/00027.d1d0f97e096fe08fc80a4939355759e7,1) Fight The Risk of Cancer!\n\nhttp://www.adc...,spam
emails/emails/spam/00197.5a921df53d215a60d557c68754559e93,THE ULTIMATE WAY TO WORK FROM HOME \n\nTHE BES...,spam
emails/emails/spam/00018.5b2765c42b7648d41c93b9b27140b23a,"DEAR FRIEND,I AM MRS. SESE-SEKO WIDOW OF LATE...",spam
emails/emails/spam/00408.22230b84aee00e439ae1938e025d5005,"\n\n<html>\n\n<body bgcolor=""#FFFFFF"">\n\n<TAB...",spam
emails/emails/spam/00073.8dcd40346d48c69a9e075e935395e96d,Dear Sir / Madam\n\n\n\nIf you are fed up of b...,spam
emails/emails/spam/00395.f9df5b3574ef5ba6143c08a1fa301886,<html>\n\n\n\n<head>\n\n<title>Home Page</titl...,spam
emails/emails/spam/00042.3e934ba4075f82283d755174d2642b76,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr...",spam


In [11]:
# Cria um vetorizador que transforma os textos em vetores de contagem de palavras (Bag of Words)
vectorizer = CountVectorizer()

# Aplica o vetorizador aos e-mails, transformando a coluna 'message' em uma matriz esparsa de contagens
counts = vectorizer.fit_transform(data['message'].values)

# Cria o classificador Naive Bayes (Multinomial), que é adequado para dados de contagem como texto
classifier = MultinomialNB()

# Extrai as classes ('spam' ou 'ham') dos dados, que serão os alvos do modelo
targets = data['class'].values

# Treina o classificador usando os vetores de contagem como entrada e as classes como saída
classifier.fit(counts, targets)

MultinomialNB()

In [13]:
# Usa o vetorizador treinado (CountVectorizer) para transformar os textos do conjunto de teste em vetores numéricos
# Isso converte os e-mails do dataTest em representações que o classificador entende
predictions = classifier.predict(vectorizer.transform(dataTest['message'].values))

# Exibe as previsões feitas pelo classificador
# O resultado será uma lista com os rótulos previstos: 'spam' ou 'ham' para cada e-mail do conjunto de teste
predictions

array(['spam', 'spam', 'spam', 'spam', 'ham', 'spam', 'spam', 'spam',
       'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam',
       'ham', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam',
       'spam', 'spam', 'spam', 'spam', 'ham', 'spam', 'spam', 'spam',
       'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam',
       'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam',
       'spam', 'spam', 'ham', 'spam', 'spam', 'spam', 'spam', 'spam',
       'spam', 'spam', 'spam', 'ham', 'spam', 'ham', 'spam', 'spam',
       'spam', 'ham', 'ham', 'spam', 'spam', 'ham', 'spam', 'spam',
       'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'ham', 'spam',
       'spam', 'spam', 'ham', 'spam', 'spam', 'spam', 'spam', 'ham',
       'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam',
       'spam', 'spam', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ha